# Εργαστηριακή Άσκηση 3 : Βαθία μάθηση


Σε αυτή την εργαστηριακή άσκηση θα επικεντρωθούμε στην μοντελοποίηση ενός νευρωνικού δικτύου παραγωγής λεκτικών περιγραφών από εικόνες (Image Captioning). Το dataset που θα χρησιμοποιήσουμε είναι το "flickr30k-images-ecemod", μια παραλλαγή του flick30k για το μάθημά μας. Στo flickr30k-images-ecemod κάθε εικόνα έχει 5 captions. Συγκεκριμένα, τα δεδομένα του flickr30k-images-ecemod είναι τα εξής : 
* ένας φάκελος "image_dir" με 31.783 εικόνες από το Flickr
* ένα αρχείο "train_captions.csv" με 148.915 captions για τις εικόνες του "image_dir"
* ένα αρχείο "test_images.csv" με 2.000 ονόματα εικόνων από το "imag_dir" που δεν έχουν captions εκπαίδευσης

In [ ]:
########################
""" Import Libraries """
########################

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import os
import collections
import random
import re
from glob import glob
import time
import json
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
#from kaggle_datasets import KaggleDatasets
#import efficientnet.tfkeras as efn

######################################################
""" Useless Functions to check stuff in text files """
######################################################

#def to show image
def PlotarismaIMG(eikona, titlos):
    plt.imshow(eikona) 
    plt.title(titlos)
    plt.show()
    return None

#def to show image in grayscale
def grayIMG(eikona, titlos):
    plt.imshow(eikona, cmap = 'gray') 
    plt.title(titlos)
    plt.show()
    return None

#def to show image in grayscale
def grayRevIMG(eikona, titlos):
    plt.imshow(eikona, cmap = 'gray_r') 
    plt.title(titlos)
    plt.show()
    return None

def axristi_diaxorisi():
    print("o----------------------o",
          "o----------------------o",
          "o----------------------o",
          "o----------------------o",
          "o----------------------o","\n")
    return None

Στο επόμενο κελί κατεβάζουμε τα απαιτούμενα datasets για την μοντελοποίηση του δικτύου μας, δηλαδή τις εικόνες του dataset (31.782 + 1 read_me.txt) στον φάκελο _image_dir_, τα _train_captions.csv_ που περιέχουν paths για τις εικόνες μέσα στο φάκελο _image_dir_ με τα αντίστοιχα captions τους, και τα _test_images.csv_ που περιέχουν paths για τις εικόνες μέσα στο φάκελο _image_dir_ , οι οποίες δεν έχουν caption

In [ ]:
# Download image files
image_zip = tf.keras.utils.get_file('flickr30k-images-ecemod.zip',
                                    cache_subdir = os.path.abspath('.'),
                                    origin='https://spartacus.1337.cx/flickr-mod/flickr30k-images-ecemod.zip',
                                    extract=True)

image_dir_folder = os.path.dirname(image_zip)+"/image_dir"
os.remove(image_zip)

# Download train captions file
# every train img has 5 captions
train_captions_file = tf.keras.utils.get_file('train_captions.csv',
                                           cache_subdir=os.path.abspath('.'),
                                           origin='https://spartacus.1337.cx/flickr-mod/train_captions.csv',
                                           extract=False)

# Download test files list
test_images_file = tf.keras.utils.get_file('test_images.csv',
                                           cache_subdir=os.path.abspath('.'),
                                           origin='https://spartacus.1337.cx/flickr-mod/test_images.csv',
                                           extract=False)

In [ ]:
# put all images paths in one list
image_dir_files = os.listdir(image_dir_folder)
Images_Paths = []
for i in range(0,len(image_dir_files)):
    if(image_dir_files[i][-3:]=="jpg"):
        Images_Paths.append("/kaggle/working/image_dir/"+image_dir_files[i])


# Ελάττωση πλήθους δεδομένων

Στα επόμενα κελιά δημιουργούνται δύο συναρτήσεις, οι οποίες θα βοηθήσουν στην επιτάχυνση της διαδικασίας της εκπαίδευσης, ενώ παράλληλα κρίνονται αναγκαίες δεδομένου ότι εργαζόμαστε στο kaggle και η παρεχόμενη μνήμη RAM είναι περιορισμένη. Αναλυτικά, οι συναρτήσεις λειτουργούν ως εξής : 
* **group_captions** δέχεται ως είσοδο το train.csv αρχείο και ομαδοποιεί τα captions κάθε εικόνας, επιστρέφοντας στην έξοδο ένα dictionary με keys τα paths για κάθε εικόνα και values τα 5 captions που αντιστοιχούν σε κάθε εικόνα
* **group_imgs** δέχεται ως είσοδο τον φάκελο _image_dir_ και επιστρέφει ένα dictionary με keys τα paths για κάθε εικόνα και values τις εικόνες σε "αναγνώσιμη" μορφή με χρήση της _PIL.Image_

In [ ]:
######################
""" Group Captions """
######################

def group_captions(FILE):
    
    with open(FILE, "r") as f:
        
        lines = f.readlines()
    
        # Get the ids
        ids = []
        for line in lines:
            ids.append(line.split(".")[0])
    
        # Get the captions
        captions = []
        for line in lines:
            captions_helper = line.split("|")[2]
            captions_helper2 = captions_helper.rstrip("\n")
            captions.append(captions_helper2)
    
        #checker = ids[0]
        image_path_to_caption = collections.defaultdict(list)
        for iD in range(0,len(ids)):
            caption = "<start> "+captions[iD]+" <end>"
            path_2_img = "/kaggle/working/image_dir/"+ids[iD]+".jpg"
            image_path_to_caption[path_2_img].append(caption)
    
        #print(list(image_path_to_caption.keys())[0])
        #print(list(image_path_to_caption.values())[0])
    
    return image_path_to_caption

In [ ]:
image_path_to_train_caption = group_captions(train_captions_file)

In [ ]:
####################
""" Group Images """
####################

def group_imgs(folder_path):
    
    jpgs = []
    jpg_names = os.listdir(folder_path)
    image_id = [] 
    for jpg_name in jpg_names:
        if jpg_name[-3:]== "jpg":
            jpgs.append(Image.open(folder_path+"/"+jpg_name))
            image_id.append(jpg_name.split(".")[0])
    
    image_path_to_jpg = collections.defaultdict(list)
    for iD in range(0,len(image_id)):
        path_2_img = "/kaggle/working/image_dir/"+image_id[iD]+".jpg"
        image_path_to_jpg[path_2_img].append(jpgs[iD])
    
    #print(list(image_path_to_jpg.keys())[0])
    #print(list(image_path_to_jpg.values())[0])
    
    return image_path_to_jpg

In [ ]:
image_dir_folder = os.path.dirname(image_zip)+"/image_dir"
image_path_to_jpg2 = group_imgs(image_dir_folder)
#print(list(image_path_to_jpg2.keys())[:5])

In [ ]:
#######################################
""" Show Images with their captions """
#######################################

def IMG_captions(File_name, path_2_jpg=image_path_to_jpg2, path_2_caption=image_path_to_train_caption):
    path_2_image_dir = "/kaggle/working/image_dir/"
    PlotarismaIMG(path_2_jpg[path_2_image_dir+File_name][0], " ")
    print("The captions of the image "+File_name+" are : \n")
    print(path_2_caption[path_2_image_dir+File_name])
    return None

In [ ]:
IMG_captions("_124857180.jpg")

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.keras.layers.Resizing(299, 299)(img)
    ## choose the suitable preprocessing for CNN
    #img = tf.keras.applications.inception_v3.preprocess_input(img) 
    img = tf.keras.applications.xception.preprocess_input(img)
    return img, image_path

In [ ]:
img1,_ = load_image("/kaggle/working/image_dir/_1000070808.jpg")
PlotarismaIMG(img1, " ")

# Δημιουργία Keras μοντέλου αρχιτεκτονικής InceptionV3 - Xception

Στο επόμενο κελί δημιουργούμε ένα _tf.keras_ μοντέλο, όπου το τελευταίο επίπεδο (layer) είναι το τελευταίο συνελκτικό επίπεδο στην InceptionV3 αρχιτεκτονική. Οι διαστάσεις του output_layer θα είναι 8x8x2048 για InceptionV3 ή 10x10x1280 για EfficientNetB1. Το τελευταίο συνελκτικό επίπεδο το χρησιμοποιούμε γιατί η λογική μάθησης του μοντέλου είναι βασισμένη στην attention. Ακολούθως γίνονται τα παρακάτω βήματα, 
* κάθε εικόνα "περνάει" από το δίκτυο και το διάνυσμα που προκύπτει αποθηκεύεται σε ένα dictionary (image_name --> feature_vector)
* αυτό το dictionary αποθηκεύεται στον δίσκο

In [ ]:
#########################################
""" Initialize InceptionV3 - Xception """
#########################################

image_model = tf.keras.applications.Xception(include_top=False,weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output
image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
# check the model that we've mad
#print(image_model.summary())
axristi_diaxorisi()
print("Information of the CNN output")
print(image_model.layers[-1], image_model.layers[-1].output.shape)
#print(image_model)
axristi_diaxorisi()
print("New_input \n",new_input)
axristi_diaxorisi()
print("Hidden_layer \n",hidden_layer)
axristi_diaxorisi()
print("Image_features_extract_model \n",image_features_extract_model)

# Λήψη Δείγματος Δεδομένων

Στο επόμενο κελί επιλέγεται ένας συγκεκριμένος αριθμός εικόνων _Samples_ , ο οποίος θα αποτελέσει το σύνολο εκπαίδευσης, μιας και στο kaggle η παρεχόμενη μνήμη RAM είναι περιορισμένη. Πρέπει να σημειωθεί ότι αυτή η διαδικασία θα μειώσει την επιτυχία αποφάσης ακριβέστερου caption, όμως θα επιταχύνει τη διαδικασία. 

Ο img_name_train_vector περιλαμβάνει τα paths των εικόνων που θα χρησιμοποιήσουμε για την εκπαίδευση του μοντέλου.Το length του είναι 30.000 και προκύπτει ως το πλήθος των εικόνων που επιλέξαμε για την εκπαιδευση , δηλαδή 6.000 ,επί το πλήθος των περιγραφών για την κάθε εικόνα οι οποίες όπως αναφέραμε προηγουμένως είναι 5.Συνεπώς αν εκτυπώσουμε τις 5 πρώτες τιμές του img_name_train_vector θα δούμε ότι τυπώνουν το ίδιο path.

Αντίστοιχα ο πίνακας captions_train έχει πάλι length 30.000 και περιλαμβάνει όλα τα captions για όλες τις εικόνες που θα χρησιμοποιήσουμε για την εκπαίδευση του μοντέλου.Αν χρησιμοποιήσουμε το indexing ενός συγκεκριμένου caption στο img_name_train_vector μπορούμε να δούμε σε ποια εικόνα αναφέρεται.

In [ ]:
#############################
""" Get The Unique Images """
#############################

def Get_Images_Captions_Vectors(img_path_dict, Samples):
    
    # Get unique images by following the TENSORFLOW TUTORIAL
    Image_Paths = list(img_path_dict.keys())[:Samples]
    captions_vector = []
    img_name_vector = []

    for image_path in Image_Paths:
        Captions_InList = img_path_dict[image_path]
        captions_vector.extend(Captions_InList)
        img_name_vector.extend([image_path] * len(Captions_InList))

    print(captions_vector[0])
    PlotarismaIMG(Image.open(img_name_vector[0])," ")    
    
    #Encoded = sorted(set(img_name_vector))
    
    return img_name_vector, captions_vector

In [ ]:
img_name_train_vector, captions_train = Get_Images_Captions_Vectors(image_path_to_train_caption, 6000)

# Προεπεξεργασία και tokenization των captions

Στο επομένο κελί γίνεται η προεπεξεργασία των captions με σπάσιμο σε λέξεις-tokens. Αυτά το tokens θα συντελέσουν στην κατασκευή ενός dictionary 5000 στοιχείων, όπου αντιστοιχεί κάθε λέξη στον index της στο vocabulary. Τέλος δημιουργούνται τα word-to-index και index-to-word mappings για να οπτικοποιηθούν τα αποτελέσματα.

Αρχικά κρατάω τα captions με μήκος λέξεων στο range(22,46) εφόσον τα πολύ μικρά ή τα πολύ μεγάλα captions δεν μας δίνουν ωφέλιμη πληροφορία στην εκπαίδευση.

Επιπλέον αυξάνω το μέγεθος του vocabulary στις 10000 λέξεις.

Τέλος, στην standardize προσθέτω ένα ακόμα φίλτρο lemmatization και επισης αφαιρώ από τα captions τις λέξεις που έχουν μήκος χαρακτήρων μικρότερο του 3.

In [ ]:
############################################
""" Preprocess and tokenize the captions """
############################################

captions_train_optim=[]
img_name_train_vector_optim=[]
id_2_check = img_name_train_vector[0]
is_not_0 = []
for idx, caption in enumerate(captions_train):
    #id_2_caption[idx] = img_name_train_vector[idx]
    words=caption.split(' ')
    #το μήκος της λέξης προκύπτει ως len(words)-4 λέξεις οι οποίες είναι το '<start>','<end>','.' και ένα κενό
    if(len(words)<=50 and len(words)>=18):
        captions_train_optim.append(caption)   
        img_name_train_vector_optim.append(img_name_train_vector[idx])
        if(id_2_check==img_name_train_vector[idx]):
            is_not_0.append(False)
    
    if(len(words)>50 or len(words)<18):
        is_not_0.append(True)
    
    if(((idx+1) % 5) == 0):
        if(all(is_not_0)):
            is_not_0 = []
            captions_train_optim.append(caption)   
            img_name_train_vector_optim.append(img_name_train_vector[idx])
            if((idx+1)==len(captions_train)):
                id_2_check=img_name_train_vector[idx]
        else:
            is_not_0 = []
            if((idx+1)==len(captions_train)):
                id_2_check=img_name_train_vector[idx]
        
print(len(captions_train))
print(len(captions_train_optim))

In [ ]:
"""Lemmatization of a caption """
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()


def Lem(caption):
    word_list=caption.split(' ')
    lem_output=' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lem_output

In [ ]:
"""Keep only the words with lentgh<=3"""
captions_new=[]
for caption in captions_train_optim:
    lista=[]
    word_list=caption.split(' ')
    for idx in range(2,len(word_list)-2):
        word=word_list[idx]
        if len(word)>=3:
            lista.append(word)
    sentence=' '.join([w for w in lista])
    sentences='<start>  ' + sentence + ' <end>'
    captions_new.append(sentences)
        
            
            

In [ ]:
print(captions_new[:5])
print(captions_train_optim[0])

In [ ]:
"""Lemmatization in the whole dataset captions_train_optim """
captions_train_lem=captions_new
for caption in captions_train_optim:
    captions_train_lemi.append(Lem(caption))

In [ ]:
from nltk.stem import WordNetLemmatizer

caption_dataset = tf.data.Dataset.from_tensor_slices(captions_train_lem)

# We will override the default standardization of TextVectorization to preserve
# "<>" characters, so we preserve the tokens for the <start> and <end>.
def standardize(inputs):
    inputs = tf.strings.lower(inputs)
    return tf.strings.regex_replace(inputs,r"!\"#$%&\(\)\*\+.,-/:;=?@\[\\\]^_`{|}~", "")

# Max word count for a caption.
max_length =45

# Use the top 5000 words for a vocabulary.
vocabulary_size =10000
tokenizer = tf.keras.layers.TextVectorization(max_tokens=vocabulary_size,standardize=standardize,output_sequence_length=max_length)

# Learn the vocabulary from the caption data.
tokenizer.adapt(caption_dataset)

# Create the tokenized vectors
cap_vector = caption_dataset.map(lambda x: tokenizer(x))

# Create mappings for words to indices and indicies to words.
word_to_index = tf.keras.layers.StringLookup(mask_token="",vocabulary=tokenizer.get_vocabulary())
index_to_word = tf.keras.layers.StringLookup(mask_token="",vocabulary=tokenizer.get_vocabulary(),invert=True)

# Caching τα διανύσματα από το InceptionV3

Στο επόμενο κελί τα διανύσματα - χαρακτηριστικά που προέκυψαν από το μοντέλο keras αρχιτεκτονικής InceptionV3 αποθηκεύονται προσωρινά στην Cache.

In [ ]:
##################################################################
""" Caching the features extracted from InceptionV3 - Xception """
##################################################################

encode_train = sorted(set(img_name_train_vector_optim))

feature_dict = {}

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

for img, path in image_dataset:
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,(batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        path_of_feature = p.numpy().decode("utf-8")
        np.save(path_of_feature, bf.numpy())
        feature_dict[path_of_feature] =  bf.numpy()

# Διαχωρισμός Δεδομένων σε train, validation και test set

Στο παρακάτω κελί υλοποιείται ο διαχωρισμός σε train και dev set με καθοδήγηση από το tutorial του TensorFlow ["Image captioning with visual attention"](https://www.tensorflow.org/tutorials/text/image_captioning) .

In [ ]:
#########################################
""" Split the data into train and dev """
#########################################

img_to_cap_vector = collections.defaultdict(list)
for img, cap in zip(img_name_train_vector, cap_vector):
    img_to_cap_vector[img].append(cap)

# Create training and validation sets using an 80-20 split randomly.
img_keys = list(img_to_cap_vector.keys())
#random.shuffle(img_keys)

slice_index = int(len(img_keys)*0.8)
img_name_train_keys, img_name_val_keys = img_keys[:slice_index], img_keys[slice_index:]

img_name_train = []
cap_train = []
for imgt in img_name_train_keys:
    capt_len = len(img_to_cap_vector[imgt])
    img_name_train.extend([imgt] * capt_len)
    cap_train.extend(img_to_cap_vector[imgt])

img_name_val = []
cap_val = []
for imgv in img_name_val_keys:
    capv_len = len(img_to_cap_vector[imgv])
    img_name_val.extend([imgv] * capv_len)
    cap_val.extend(img_to_cap_vector[imgv])
    
#Normalization in batch_size

img_name_train = img_name_train[:len(img_name_train)-(len(img_name_train) % 100)]
cap_train = cap_train[:len(cap_train)-(len(cap_train) % 100)]
img_name_val = img_name_val[:len(img_name_val)-(len(img_name_val) % 100)]
cap_val = cap_val[:len(cap_val)-(len(cap_val) % 100)]

In [ ]:
print(len(img_name_val))
print(cap_train[0])

# Δημιουργία tf.data dataset για εκπαίδευση

In [ ]:
#############################################
""" Create a tf.data dataset for training """
#############################################

# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 100
BUFFER_SIZE = 1000
embedding_dim = 100
units = 512
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
features_shape = 2048
attention_features_shape = 100

# Load the numpy files
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8')+'.npy')
    return img_tensor, cap

###------------------- train - dataset ------------------------###
train_dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
train_dataset = train_dataset.map(lambda item1, item2: tf.numpy_function(map_func,[item1,item2],[tf.float32,tf.int64]),
                                  num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle and batch
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)


###------------------- val - dataset ------------------------###
val_dataset = tf.data.Dataset.from_tensor_slices((img_name_val, cap_val))

# Use map to load the numpy files in parallel
val_dataset = val_dataset.map(lambda item1, item2: tf.numpy_function(map_func,[item1,item2],[tf.float32,tf.int64]),
                              num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle and batch
val_dataset = val_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Embeddings

In [ ]:
"""Embeddings through gensim"""
#!pip install -U gensim
import pickle
import gensim
import gensim.downloader
a=list(gensim.downloader.info()['models'])
print(a)



In [ ]:
#################################################################################
""" load the glove_wiki_gigaword models and dump them using the pickle method """
#################################################################################

#model_50=gensim.downloader.load('glove-wiki-gigaword-50')
#pickle.dump(model_50, open("glove_wiki_50", 'wb'))

model_100=gensim.downloader.load('glove-wiki-gigaword-100')
pickle.dump(model_100, open("glove_wiki_100", 'wb'))

#model_200=gensim.downloader.load('glove-wiki-gigaword-200')
#pickle.dump(model_200, open("glove_wiki_200", 'wb'))

#model_300=gensim.downloader.load('glove-wiki-gigaword-300')
#pickle.dump(model_300, open("glove_wiki_300", 'wb'))


Χρησιμοποιούμε το μοντέλο με τα 100 embeddings καθώς τα ανώτερα μοντέλα των 200 και 300 εξαντλούν την μνήμη RAM του Kaggle.

In [ ]:
loaded_model_100 = pickle.load(open('glove_wiki_100', 'rb'))

In [ ]:
"""Create embedding matrix"""
embedding_matrix = np.zeros((10000, embedding_dim))
for idx, word in enumerate(tokenizer.get_vocabulary()):
    try:
        embedding_vector = loaded_model_100[word]
        embedding_matrix[idx] = embedding_vector
    except KeyError:
        vec = np.zeros(embedding_dim)
        embedding_matrix[idx] = vec

# Νευρωνικό Μοντέλο

Ο τρόπος λειτουργίας του νευρωνικού δικτύου βασίζεται στο [Show, Attend and Tell](https://arxiv.org/pdf/1502.03044.pdf). Συγκεκριμένα ακολουθεί τα παρακάτω βήματα: 
* τα χαρακτηριστικά εξάγονται από το τελευταίο συνελκτικό επίπεδο του InceptionV3 δίνοντας ένα διάνυσμα (8,8,2048)
* τα διανύσματα αυτά αναδιατάσσονται σε διαστάσεις (64,2048)
* τα διανύσματα αυτά μετά περνάνε στο CNN Encoder, ο οποίος αποτελείται ένα fully connected layer
* ύστερα το RNN, σε αυτή την περίπτωση το GRU, σκανάρει αναδρομικά την εικόνα για προβλέψει την επόμενη λέξη

In [ ]:
##############
""" Models """
##############

class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

        # hidden shape == (batch_size, hidden_size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # attention_hidden_layer shape == (batch_size, 64, units)
        attention_hidden_layer = (tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))

        # score shape == (batch_size, 64, 1)
        # This gives you an unnormalized score for each image feature.
        score = self.V(attention_hidden_layer)

        # attention_weights shape == (batch_size, 64, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)
        # apply Dropout
        self.dropout = tf.keras.layers.Dropout(0.15)
        

    def call(self, x):
        x = self.fc(x)
        x = self.dropout(x)
        x = tf.nn.relu(x)
        return x

class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size, gru_OR_lstm):
        super(RNN_Decoder, self).__init__()
        self.units = units
        
        # new Embedding layer
        
        self.embedding_layer = tf.keras.layers.Embedding(10000,
                                                         embedding_dim,
                                                         weights=[embedding_matrix],
                                                         trainable=False)
        
        # Declaration of GRU
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        
        # Declaration of LSTM
        self.lstm = tf.keras.layers.LSTM(self.units,
                                        return_sequences=True,
                                        return_state=True,
                                        recurrent_initializer='glorot_uniform')
        
        # boolean to decide which model
        self.gru_OR_lstm = gru_OR_lstm
        
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)
        
        # apply Dropout
        self.dropout = tf.keras.layers.Dropout(0.15)
        
        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        # defining attention as a separate model
        context_vector, attention_weights = self.attention(features, hidden)
        

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        #x = self.embedding(x)
        x = self.embedding_layer(x)
        

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        if(self.gru_OR_lstm=="GRU"):
            # passing the concatenated vector to the GRU
            output, state = self.gru(x)
        
        if(self.gru_OR_lstm=="LSTM"):
            # passing the concatenated vector to the GRU
            output, state, carry_state = self.lstm(x)
            
        # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(output)
        
        x = self.dropout(x)
        
        # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

        # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)
        
        x = self.dropout(x)
        
        return x, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))
        
        

In [ ]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, tokenizer.vocabulary_size(), "GRU")

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# Checkpoint

In [ ]:
##################
""" Checkpoint """
##################


checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,decoder=decoder,optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)



# Training

In [ ]:
################
""" Training """
################

# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

@tf.function
def train_step(img_tensor, target):
    loss = 0
    
    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])
    
    dec_input = tf.expand_dims((loaded_model_100.get_vector("#")) * target.shape[0],1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)
            #print("predictions",predictions)
            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

start_epoch = 0
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(train_dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = batch_loss.numpy()/int(target.shape[1])
            print(f'Epoch {epoch+1} Batch {batch} Loss {average_batch_loss:.4f}')
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()

    print(f'Epoch {epoch+1} Loss {total_loss/num_steps:.6f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

# Image Captioning Evaluation

Στα παρακάτω κελιά υλοποιείται η αξιολόγηση των captions που προβλέπει το δίκτυο με δίκη μας κρίση, δηλαδή με τις παρακάτω συναρτήσεις προκύπτουν στην έξοδο το αληθινό caption και το προβλεπόμενο από το δίκτυο και με δική μας κρίση αποφασίζουμε αν ήταν επιτυχές το caption για την αντίστοιχη εικόνα. 

Δοκιμάσαμε στο evaluation να χρησιμοποιήσουμε beam_search για τα prediction_captions . O κώδικας που χρησιμοποιήσαμε είναι ο ακόλουθος:

def beam_evaluation(image):
    
    max_length=20
    attention_plot = np.zeros((max_length, attention_features_shape))
    beam_index=3
    hidden = decoder.reset_state(batch_size=1)
    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0],
                                                 -1,
                                                 img_tensor_val.shape[3]))
    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([word_to_index(tf.constant('<start>'))], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input,
                                                         features,
                                                         hidden)
        prob=[]
        zeygos=[]
        predictions=predictions.numpy()
        predictions=predictions[0]
        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
        
        Beam_search
        predicted_id = tf.argsort(predictions, axis=1)[0].numpy()
        predicted_id=predicted_id[:beam_index]
        
        for pred in predicted_id:
            predicted_word =
                      tf.compat.as_text(index_to_word(tf.constant(pred)).numpy())
            result.append(predicted_word)
            
            probabilliti=predictions[pred]
            prob.append(probabilities)
            probabilliti+=probability
            zeygos.append((predicted_word,probabilliti))
           if predicted_word == '<end>':
               return result, attention_plot

           dec_input = tf.expand_dims([predicted_id], 0)
        result=np.argmax(result)
    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot




Παρόλα αυτά παρατηρήσαμε στα captions ότι ενώ οι αρχικές λέξεις που προέβλεπε ήταν καλές στην συνέχεια 'χανόταν' με αποτέλεσμα το bleu score να είναι πολύ χαμηλό.(Προφανώς η υλοποίηση μας κάπου μπάζει). Αντ'αυτού χρησιμοποιήσαμε Greedy Search.

In [ ]:
def evaluate(image):
    max_length=20
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0],
                                                 -1,
                                                 img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([word_to_index(tf.constant('<start>'))], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input,
                                                         features,
                                                         hidden)
        
        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
        
        #Greedy_search
        predicted_id = tf.math.argmax(predictions, axis=1)[0].numpy()
        
        predicted_word = tf.compat.as_text(index_to_word(tf.constant(predicted_id)).numpy())
        result.append(predicted_word)

        if predicted_word == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for i in range(len_result):
        temp_att = np.resize(attention_plot[i], (8, 8))
        grid_size = max(int(np.ceil(len_result/2)), 2)
        ax = fig.add_subplot(grid_size, grid_size, i+1)
        ax.set_title(result[i])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [ ]:
def Real_Pred_Caption_Printer(rid):
    # captions on the validation set
    image = img_name_val[rid]
    real_caption =image_path_to_train_caption[image]
    #real_caption = ' '.join([tf.compat.as_text(index_to_word(tf.constant(i)).numpy()) for i in cap_val[rid] if i not in [0]])
    result, attention_plot = evaluate(image)
    print('len',len(result))

    print('Real Caption:', real_caption)
    print('Prediction Caption:', ' '.join(result))
    plot_attention(image, result, attention_plot)
    
    return real_caption, result

In [ ]:
### random image captioning ###
RID = np.random.randint(0, len(img_name_val))
print("The id of the image is: ",img_name_val[RID])
_,_ = Real_Pred_Caption_Printer(RID)



# Cherry Picking 
Στο επόμενο κελί δίνονται κάποια ids καλών παραδειγμάτων image captioning από το μοντέλο, που δημιουργήσαμε μέχρι στιγμής.

In [ ]:
len(img_name_val)

In [ ]:
cherry_list = [100, 506, 1218, 2000, 1254]

cherry_ref = []
cherry_hypo = []
for cherry in cherry_list:
    cherry_ref_helper,cherry_hypo_helper = Real_Pred_Caption_Printer(cherry)
    #cherry_ref.append(cherry_ref_helper)
   # cherry_hypo.append(cherry_hypo_helper)
    #axristi_diaxorisi()

# Nir Picking

Στο επόμενο κελί δίνονται κάποια ids καλών παραδειγμάτων image captioning από το μοντέλο, που δημιουργήσαμε μέχρι στιγμής.

In [ ]:
nir_list = [3005, 368, 2456, 1308, 1725]

nir_ref = []
nir_hypo = []
for nir in nir_list:
    nir_ref_helper,nir_hypo_helper = Real_Pred_Caption_Printer(nir)
    nir_ref.append(nir_ref_helper)
    nir_hypo.append(nir_hypo_helper)
    axristi_diaxorisi()

# Validation set's references & hypothesis

In [ ]:
###############################################################
""" Validation Set passed through Real_Pred_Caption_Printer """
###############################################################


val_ref = []
val_hypo = []
for inv_id,inv in enumerate(img_name_val):
    val_hypo_helper = evaluate(inv)[0]
    val_hypo.append(val_hypo_helper)
    val_ref_helper = ' '.join([tf.compat.as_text(index_to_word(tf.constant(i)).numpy()) for i in cap_val[inv_id] if i not in [0]])
    val_ref.append(val_ref_helper)
    


# Αξιολόγηση της ποιότητας του captioning

Για την εκτίμηση της ποιότητας του παραγόμενου captioning θα χρησιμοποιηθεί το BLEU (Bilingual Evaluation Understudy) score, μεταξύ hypothesis(το caption του νευρωνικού) και references(το αληθινό caption). Το BLEU είναι μια μετρική για αξιολογήσης μοντέλων παραγωγής (και μετάφρασης) κειμένων. 

Συγκεκριμένα, είναι ένας αριθμός μεταξύ του 0 και του 1, ο οποίος αντιπροσωπεύει την ομοιότητα των παραγόμενων κειμένων από τα διάφορα μοντέλα σε σύγκριση με κάποια καλά captions (ή μεταφράσεις αντιστοίχως). Τιμή BLEU ίση με το 0 σημαίνει ότι δεν υπήρχε καμία επικάλυψη μεταξύ των λέξεων προβλεπόμενου και πραγματικού κειμένου. Αντιθέτως, τιμή BLEU ίση με τη 1 σημαίνει ότι υπήρχε πλήρης επικάλυψη μεταξύ των λέξεων προβλεπόμενου και πραγματικού κειμένου. Έχει αποδειχθεί ότι το BLEU score σχετίζεται σημαντικά με την ανθρώπινη εκτίμηση για την επιτύχια captioning (και μετάφρασης κειμένου).

Για τον υπολογισμού των BLEU scores η NLTK στο [nltk.translate.bleu_score](https://www.nltk.org/_modules/nltk/translate/bleu_score.html) παρέχει τις απαραίτητες συναρτήσεις : 
* την _sentence_bleu_ , για την αξιολόγηση του captioning ενός μεμονωμένου παραδείγματος
* την _corpus_bleu_ , για την αξιολόγηση του captioning ολόκληρου set εικόνων

Σε όλες τις περιπτώσεις χρησιμοποιούνται οι εξής παραμέτροι: _weights=(0.4, 0.3, 0.2, 0.1)_ και _smoothing_function=SmoothingFunction().method1_

In [ ]:
#################################
""" BLEU score implementation """
#################################

import math
import sys
import warnings
from collections import Counter
from fractions import Fraction
from nltk.util import ngrams


def sentence_bleu(references,hypothesis,weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=None,auto_reweigh=False):
    
    """ PARAMETER ANALYSIS
        * param references: reference sentences
        * type references: list(list(str))
        * param hypothesis: a hypothesis sentence
        * type hypothesis: list(str)
        * param weights: weights for unigrams, bigrams, trigrams and so on (one or a list of weights)
        * type weights: tuple(float) / list(tuple(float))
        * param smoothing_function:
        * type smoothing_function: SmoothingFunction
        * param auto_reweigh: Option to re-normalize the weights uniformly.
        * type auto_reweigh: bool
        * return: The sentence-level BLEU score. Returns a list if multiple weights were supplied.
        * rtype: float / list(float
    """
    
    The_Return = corpus_bleu([references], [hypothesis], weights, smoothing_function, auto_reweigh)
    
    return The_Return

def corpus_bleu(list_of_references,hypotheses,weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=None,auto_reweigh=False):
    
    """ PARAMETER ANALYSIS
        * param list_of_references: a corpus of lists of reference sentences, w.r.t. hypotheses
        * type list_of_references: list(list(list(str)))
        * param hypotheses: a list of hypothesis sentences
        * type hypotheses: list(list(str))
        * param weights: weights for unigrams, bigrams, trigrams and so on (one or a list of weights)
        * type weights: tuple(float) / list(tuple(float))
        * param smoothing_function:
        * type smoothing_function: SmoothingFunction
        * param auto_reweigh: Option to re-normalize the weights uniformly.
        * type auto_reweigh: bool
        * return: The corpus-level BLEU score.
        * rtype: float
    """
    # Before proceeding to compute BLEU, perform sanity checks.

    p_numerators = Counter()  # Key = ngram order, and value = no. of ngram matches.
    p_denominators = Counter()  # Key = ngram order, and value = no. of ngram in ref.
    hyp_lengths, ref_lengths = 0, 0

    assert len(list_of_references) == len(hypotheses), (
        "The number of hypotheses and their reference(s) should be the " "same "
    )

    try:
        weights[0][0]
    except TypeError:
        weights = [weights]
    max_weight_length = max(len(weight) for weight in weights)

    # Iterate through each hypothesis and their corresponding references.
    
    for references, hypothesis in zip(list_of_references, hypotheses):
        
        # For each order of ngram, calculate the numerator and
        # denominator for the corpus-level modified precision.
        
        for i in range(1, max_weight_length + 1):
            p_i = modified_precision(references, hypothesis, i)
            p_numerators[i] += p_i.numerator
            p_denominators[i] += p_i.denominator

        # Calculate the hypothesis length and the closest reference length.
        # Adds them to the corpus-level hypothesis and reference counts.
        
        hyp_len = len(hypothesis)
        hyp_lengths += hyp_len
        ref_lengths += closest_ref_length(references, hyp_len)

    # Calculate corpus-level brevity penalty.
    bp = brevity_penalty(ref_lengths, hyp_lengths)

    # Collects the various precision values for the different ngram orders.
    p_n = [
        Fraction(p_numerators[i], p_denominators[i], _normalize=False)
        for i in range(1, max_weight_length + 1)
    ]

    # Returns 0 if there's no matching n-grams
    # We only need to check for p_numerators[1] == 0, since if there's
    # no unigrams, there won't be any higher order ngrams.
    if p_numerators[1] == 0:
        return 0 if len(weights) == 1 else [0] * len(weights)

    # If there's no smoothing, set use method0 from SmoothinFunction class.
    if not smoothing_function:
        smoothing_function = SmoothingFunction().method0
    # Smoothen the modified precision.
    # Note: smoothing_function() may convert values into floats;
    #       it tries to retain the Fraction object as much as the
    #       smoothing method allows.
    p_n = smoothing_function(
        p_n, references=references, hypothesis=hypothesis, hyp_len=hyp_lengths
    )

    bleu_scores = []
    for weight in weights:
        # Uniformly re-weighting based on maximum hypothesis lengths if largest
        # order of n-grams < 4 and weights is set at default.
        if auto_reweigh:
            if hyp_lengths < 4 and weight == (0.25, 0.25, 0.25, 0.25):
                weight = (1 / hyp_lengths,) * hyp_lengths

        s = (w_i * math.log(p_i) for w_i, p_i in zip(weight, p_n) if p_i > 0)
        s = bp * math.exp(math.fsum(s))
        bleu_scores.append(s)
    
    return bleu_scores[0] if len(weights) == 1 else bleu_scores


def modified_precision(references, hypothesis, n):
    
    """ PARAMETER ANALYSIS
        * param references: A list of reference translations.
        * type references: list(list(str))
        * param hypothesis: A hypothesis translation.
        * type hypothesis: list(str)
        * param n: The ngram order.
        * type n: int
        * return: BLEU's modified precision for the nth order ngram.
        * rtype: Fraction
    """
    
    # Extracts all ngrams in hypothesis
    # Set an empty Counter if hypothesis is empty.
    counts = Counter(ngrams(hypothesis, n)) if len(hypothesis) >= n else Counter()
    
    # Extract a union of references' counts.
    # max_counts = reduce(or_, [Counter(ngrams(ref, n)) for ref in references])
    max_counts = {}
    for reference in references:
        reference_counts = (
            Counter(ngrams(reference, n)) if len(reference) >= n else Counter()
        )
        for ngram in counts:
            max_counts[ngram] = max(max_counts.get(ngram, 0), reference_counts[ngram])

    # Assigns the intersection between hypothesis and references' counts.
    clipped_counts = {
        ngram: min(count, max_counts[ngram]) for ngram, count in counts.items()
    }

    numerator = sum(clipped_counts.values())
    
    # Ensures that denominator is minimum 1 to avoid ZeroDivisionError.
    # Usually this happens when the ngram order is > len(reference).
    denominator = max(1, sum(counts.values()))

    return Fraction(numerator, denominator, _normalize=False)
    
def closest_ref_length(references, hyp_len):
    
    """ PARAMETER ANALYSIS
    This function finds the reference that is the closest length to the
    hypothesis. The closest reference length is referred to as *r* variable
    from the brevity penalty formula in Papineni et. al. (2002)

        * param references: A list of reference translations.
        * type references: list(list(str))
        * param hyp_len: The length of the hypothesis.
        * type hyp_len: int
        * return: The length of the reference that's closest to the hypothesis.
        * rtype: int
    """
    ref_lens = (len(reference) for reference in references)
    closest_ref_len = min(
        ref_lens, key=lambda ref_len: (abs(ref_len - hyp_len), ref_len)
    )
    return closest_ref_len

def brevity_penalty(closest_ref_len, hyp_len):
    """ PARAMETER ANALYSIS
        * param hyp_len: The length of the hypothesis for a single sentence OR the
            sum of all the hypotheses' lengths for a corpus
        * type hyp_len: int
        * param closest_ref_len: The length of the closest reference for a single
            hypothesis OR the sum of all the closest references for every hypotheses.
        * type closest_ref_len: int
        * return: BLEU's brevity penalty.
        * rtype: float
    """
    
    if hyp_len > closest_ref_len:
        return 1
    # If hypothesis is empty, brevity penalty = 0 should result in BLEU = 0.0
    elif hyp_len == 0:
        return 0
    else:
        return math.exp(1 - closest_ref_len / hyp_len)

class SmoothingFunction:
    """
    This is an implementation of the smoothing techniques
    for segment-level BLEU scores that was presented in
    Boxing Chen and Collin Cherry (2014) A Systematic Comparison of
    Smoothing Techniques for Sentence-Level BLEU. In WMT14.
    http://acl2014.org/acl2014/W14-33/pdf/W14-3346.pdf
    """
    
    def __init__(self, epsilon=0.1, alpha=5, k=5):
        """ PARAMETER ANALYSIS
            * param epsilon: the epsilon value use in method 1
            * type epsilon: float
            * param alpha: the alpha value use in method 6
            * type alpha: int
            * param k: the k value use in method 4
            * type k: int
        """
        self.epsilon = epsilon
        self.alpha = alpha
        self.k = k
    
    def method0(self, p_n, *args, **kwargs):
        """
        No smoothing.
        """
        p_n_new = []
        for i, p_i in enumerate(p_n):
            if p_i.numerator != 0:
                p_n_new.append(p_i)
            else:
                _msg = str(
                    "\nThe hypothesis contains 0 counts of {}-gram overlaps.\n"
                    "Therefore the BLEU score evaluates to 0, independently of\n"
                    "how many N-gram overlaps of lower order it contains.\n"
                    "Consider using lower n-gram order or use "
                    "SmoothingFunction()"
                ).format(i + 1)
                warnings.warn(_msg)
                # When numerator==0 where denonminator==0 or !=0, the result
                # for the precision score should be equal to 0 or undefined.
                # Due to BLEU geometric mean computation in logarithm space,
                # we we need to take the return sys.float_info.min such that
                # math.log(sys.float_info.min) returns a 0 precision score.
                p_n_new.append(sys.float_info.min)
        
        return p_n_new
    
    def method1(self, p_n, *args, **kwargs):
        """
        Smoothing method 1: Add *epsilon* counts to precision with 0 counts.
        """
        return [
            (p_i.numerator + self.epsilon) / p_i.denominator
            if p_i.numerator == 0
            else p_i
            for p_i in p_n
        ]
    
    def method2(self, p_n, *args, **kwargs):
        """
        Smoothing method 2: Add 1 to both numerator and denominator from
        Chin-Yew Lin and Franz Josef Och (2004) ORANGE: a Method for
        Evaluating Automatic Evaluation Metrics for Machine Translation.
        In COLING 2004.
        """
        return [
            Fraction(p_n[i].numerator + 1, p_n[i].denominator + 1, _normalize=False)
            if i != 0
            else p_n[0]
            for i in range(len(p_n))
        ]
    
    def method3(self, p_n, *args, **kwargs):
        """
        Smoothing method 3: NIST geometric sequence smoothing
        The smoothing is computed by taking 1 / ( 2^k ), instead of 0, for each
        precision score whose matching n-gram count is null.
        k is 1 for the first 'n' value for which the n-gram match count is null/

        For example, if the text contains:

        - one 2-gram match
        - and (consequently) two 1-gram matches

        the n-gram count for each individual precision score would be:

        - n=1  =>  prec_count = 2     (two unigrams)
        - n=2  =>  prec_count = 1     (one bigram)
        - n=3  =>  prec_count = 1/2   (no trigram,  taking 'smoothed' value of 1 / ( 2^k ), with k=1)
        - n=4  =>  prec_count = 1/4   (no fourgram, taking 'smoothed' value of 1 / ( 2^k ), with k=2)
        """
        incvnt = 1  # From the mteval-v13a.pl, it's referred to as k.
        for i, p_i in enumerate(p_n):
            if p_i.numerator == 0:
                p_n[i] = 1 / (2 ** incvnt * p_i.denominator)
                incvnt += 1
        return p_n
    
    def method4(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        """
        Smoothing method 4:
        Shorter translations may have inflated precision values due to having
        smaller denominators; therefore, we give them proportionally
        smaller smoothed counts. Instead of scaling to 1/(2^k), Chen and Cherry
        suggests dividing by 1/ln(len(T)), where T is the length of the translation.
        """
        incvnt = 1
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        for i, p_i in enumerate(p_n):
            if p_i.numerator == 0 and hyp_len > 1:
                # incvnt = i + 1 * self.k / math.log(
                #     hyp_len
                # )  # Note that this K is different from the K from NIST.
                # p_n[i] = incvnt / p_i.denominator\
                numerator = 1 / (2 ** incvnt * self.k / math.log(hyp_len))
                p_n[i] = numerator / p_i.denominator
                incvnt += 1
        return p_n
    
    def method5(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        """
        Smoothing method 5:
        The matched counts for similar values of n should be similar. To a
        calculate the n-gram matched count, it averages the n−1, n and n+1 gram
        matched counts.
        """
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        m = {}
        # Requires an precision value for an addition ngram order.
        p_n_plus1 = p_n + [modified_precision(references, hypothesis, 5)]
        m[-1] = p_n[0] + 1
        for i, p_i in enumerate(p_n):
            p_n[i] = (m[i - 1] + p_i + p_n_plus1[i + 1]) / 3
            m[i] = p_n[i]
        return p_n
    
    def method6(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        """
        Smoothing method 6:
        Interpolates the maximum likelihood estimate of the precision *p_n* with
        a prior estimate *pi0*. The prior is estimated by assuming that the ratio
        between pn and pn−1 will be the same as that between pn−1 and pn−2; from
        Gao and He (2013) Training MRF-Based Phrase Translation Models using
        Gradient Ascent. In NAACL.
        """
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        # This smoothing only works when p_1 and p_2 is non-zero.
        # Raise an error with an appropriate message when the input is too short
        # to use this smoothing technique.
        assert p_n[2], "This smoothing method requires non-zero precision for bigrams."
        for i, p_i in enumerate(p_n):
            if i in [0, 1]:  # Skips the first 2 orders of ngrams.
                continue
            else:
                pi0 = 0 if p_n[i - 2] == 0 else p_n[i - 1] ** 2 / p_n[i - 2]
                # No. of ngrams in translation that matches the reference.
                m = p_i.numerator
                # No. of ngrams in translation.
                l = sum(1 for _ in ngrams(hypothesis, i + 1))
                # Calculates the interpolated precision.
                p_n[i] = (m + self.alpha * pi0) / (l + self.alpha)
        return p_n
    
    def method7(self, p_n, references, hypothesis, hyp_len=None, *args, **kwargs):
        """
        Smoothing method 7:
        Interpolates methods 4 and 5.
        """
        hyp_len = hyp_len if hyp_len else len(hypothesis)
        p_n = self.method4(p_n, references, hypothesis, hyp_len)
        p_n = self.method5(p_n, references, hypothesis, hyp_len)
        return p_n

In [ ]:
#declare the BLEU standard parameters
Weights = [0.4, 0.3, 0.2, 0.1]
SmoothFunc = SmoothingFunction().method1


To BLEU score για μια τυχαία εικόνα της επιλογής μας

In [ ]:
reals_1254, hypothesis_1254 = Real_Pred_Caption_Printer(1254)
bleu_1254 = sentence_bleu(reals_1254, hypothesis_1254,weights=Weights,smoothing_function=SmoothFunc,auto_reweigh=False)
print("The bleu score for 1254 is :"+str(bleu_1254))

To BLEU score για τα cherries, που επιλέχθηκαν.

In [ ]:
bleu_cherries = corpus_bleu(cherry_ref,cherry_hypo,weights=Weights,smoothing_function=SmoothFunc,auto_reweigh=False)
print("The bleu score for the cherries picked are :",bleu_cherries)

To BLEU score για τα nir, που επιλέχθηκαν.

In [ ]:
bleu_nir = corpus_bleu(nir_ref,nir_hypo,weights=Weights,smoothing_function=SmoothFunc,auto_reweigh=False)
print("The bleu score for the nir picked are :",bleu_nir)

# To BLEU score για τo Validation set

In [ ]:
bleu_val = corpus_bleu(val_ref,val_hypo,weights=Weights,smoothing_function=SmoothFunc,auto_reweigh=False)
print("The bleu score for the validation set are :",bleu_val)